# 多头注意力
:label:`sec_multihead-attention`

在实践中，给定相同的查询、键和值集合，我们可能希望我们的模型能够结合同一注意力机制的不同行为的知识，例如捕捉序列中不同范围的依赖关系（例如，短距离与长距离）。因此，允许我们的注意力机制联合使用查询、键和值的不同表示子空间可能是有益的。

为此，不是执行单一的注意力池化，而是可以使用$h$个独立学习的线性投影对查询、键和值进行变换。然后将这些$h$个投影后的查询、键和值并行地输入到注意力池化中。最后，$h$个注意力池化的输出被拼接起来，并通过另一个学习的线性投影转换以产生最终输出。这种设计被称为*多头注意力*，其中每个$h$注意力池化输出是一个*头* :cite:`Vaswani.Shazeer.Parmar.ea.2017`。使用全连接层执行可学习的线性变换，:numref:`fig_multi-head-attention` 描述了多头注意力。

![Multi-head attention, where multiple heads are concatenated then linearly transformed.](../img/multi-head-attention.svg)
:label:`fig_multi-head-attention`

In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

## 模型

在提供多头注意力的实现之前，让我们先从数学上形式化这个模型。给定一个查询 $\mathbf{q} \in \mathbb{R}^{d_q}$、一个键 $\mathbf{k} \in \mathbb{R}^{d_k}$ 和一个值 $\mathbf{v} \in \mathbb{R}^{d_v}$，每个注意力头 $\mathbf{h}_i$  ($i = 1, \ldots, h$) 计算如下：

$$\mathbf{h}_i = f(\mathbf W_i^{(q)}\mathbf q, \mathbf W_i^{(k)}\mathbf k,\mathbf W_i^{(v)}\mathbf v) \in \mathbb R^{p_v},$$

其中
$\mathbf W_i^{(q)}\in\mathbb R^{p_q\times d_q}$,
$\mathbf W_i^{(k)}\in\mathbb R^{p_k\times d_k}$,
和 $\mathbf W_i^{(v)}\in\mathbb R^{p_v\times d_v}$
是可学习参数，而
$f$ 是注意力池化，
例如
加性注意力和缩放点积注意力
在 :numref:`sec_attention-scoring-functions` 中。
多头注意力的输出
是通过
可学习参数
$\mathbf W_o\in\mathbb R^{p_o\times h p_v}$
对 $h$ 个头的拼接进行的另一个线性变换：

$$\mathbf W_o \begin{bmatrix}\mathbf h_1\\\vdots\\\mathbf h_h\end{bmatrix} \in \mathbb{R}^{p_o}.$$

基于这种设计，每个头可以关注输入的不同部分。
比简单的加权平均更复杂的函数可以被表达。

## 实现

在我们的实现中，
我们[**为多头注意力的每个头选择缩放点积注意力**]。
为了避免计算成本和参数化成本显著增长，我们设置 $p_q = p_k = p_v = p_o / h$。
注意，如果我们将线性变换
对于查询、键和值的输出数量
设为 $p_q h = p_k h = p_v h = p_o$，则 $h$ 个头可以并行计算。
在下面的实现中，
$p_o$ 通过参数 `num_hiddens` 指定。

In [2]:
class MultiHeadAttention(d2l.Module):  #@save
    """Multi-head attention."""
    def __init__(self, num_hiddens, num_heads, dropout, bias=False, **kwargs):
        super().__init__()
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.LazyLinear(num_hiddens, bias=bias)
        self.W_k = nn.LazyLinear(num_hiddens, bias=bias)
        self.W_v = nn.LazyLinear(num_hiddens, bias=bias)
        self.W_o = nn.LazyLinear(num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        # Shape of queries, keys, or values:
        # (batch_size, no. of queries or key-value pairs, num_hiddens)
        # Shape of valid_lens: (batch_size,) or (batch_size, no. of queries)
        # After transposing, shape of output queries, keys, or values:
        # (batch_size * num_heads, no. of queries or key-value pairs,
        # num_hiddens / num_heads)
        queries = self.transpose_qkv(self.W_q(queries))
        keys = self.transpose_qkv(self.W_k(keys))
        values = self.transpose_qkv(self.W_v(values))

        if valid_lens is not None:
            # On axis 0, copy the first item (scalar or vector) for num_heads
            # times, then copy the next item, and so on
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)

        # Shape of output: (batch_size * num_heads, no. of queries,
        # num_hiddens / num_heads)
        output = self.attention(queries, keys, values, valid_lens)
        # Shape of output_concat: (batch_size, no. of queries, num_hiddens)
        output_concat = self.transpose_output(output)
        return self.W_o(output_concat)

为了实现[**多个头的并行计算**]，上述`MultiHeadAttention`类使用了如下定义的两种转置方法。具体来说，`transpose_output`方法执行的是`transpose_qkv`方法操作的逆过程。

In [3]:
@d2l.add_to_class(MultiHeadAttention)  #@save
def transpose_qkv(self, X):
    """Transposition for parallel computation of multiple attention heads."""
    # Shape of input X: (batch_size, no. of queries or key-value pairs,
    # num_hiddens). Shape of output X: (batch_size, no. of queries or
    # key-value pairs, num_heads, num_hiddens / num_heads)
    X = X.reshape(X.shape[0], X.shape[1], self.num_heads, -1)
    # Shape of output X: (batch_size, num_heads, no. of queries or key-value
    # pairs, num_hiddens / num_heads)
    X = X.permute(0, 2, 1, 3)
    # Shape of output: (batch_size * num_heads, no. of queries or key-value
    # pairs, num_hiddens / num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])

@d2l.add_to_class(MultiHeadAttention)  #@save
def transpose_output(self, X):
    """Reverse the operation of transpose_qkv."""
    X = X.reshape(-1, self.num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

让我们用一个简单的例子来[**测试我们实现的**] `MultiHeadAttention` 类，其中键和值是相同的。因此，多头注意力输出的形状是（`batch_size`，`num_queries`，`num_hiddens`）。

In [4]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_heads, 0.5)
batch_size, num_queries, num_kvpairs = 2, 4, 6
valid_lens = torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
d2l.check_shape(attention(X, Y, Y, valid_lens),
                (batch_size, num_queries, num_hiddens))

## 摘要

多头注意力机制通过不同的查询、键和值的表示子空间来结合相同的注意力池化的知识。为了并行计算多头注意力的多个头，需要适当的张量操作。

## 练习

1. 在此实验中可视化多个注意力头的权重。
1. 假设我们有一个基于多头注意力训练好的模型，并且我们希望修剪不太重要的注意力头以提高预测速度。我们如何设计实验来衡量一个注意力头的重要性？

[讨论](https://discuss.d2l.ai/t/1635)